# FLAN-T5 Clinical Query Summaries
This notebook fine-tunes `google/flan-t5-base` on the clinical query summarization dataset under `data/`. It also captures baseline generations from the frozen base model and evaluates everything with ROUGE + BERTScore so you can compare against the artifacts in `gemma_2/outputs`.


## Notebook game plan
1. Install/load dependencies and set up paths.
2. Parse the provided JSON splits into flattened instruction/target rows.
3. Tokenize data for FLAN-T5 and configure the seq2seq trainer.
4. Fine-tune, evaluate, and save generated summaries plus metrics.
5. Run baseline inference with the untouched FLAN-T5-Base model and dump predictions to JSON for side-by-side comparison.


In [ ]:
!pip install -q "transformers>=4.44.0" "datasets>=2.19.0" "evaluate>=0.4.1" "accelerate>=0.30.0" sentencepiece rouge-score bert-score

print("✅ Installed core NLP libraries.")


In [ ]:
import inspect
import json
import random
from datetime import datetime
from pathlib import Path

import numpy as np
import torch
from datasets import Dataset, DatasetDict
from tqdm.auto import tqdm
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)
import evaluate

BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "data"
OUTPUT_DIR = BASE_DIR / "outputs"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "google/flan-t5-base"
MAX_INPUT_TOKENS = 1024
MAX_TARGET_TOKENS = 256
SEED = 7

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"📂 Data dir: {DATA_DIR.resolve()}")
print(f"💾 Outputs will land in: {OUTPUT_DIR.resolve()}")
print(f"🧠 Using device: {device}")


In [ ]:
def load_split(split_path: Path):
    with open(split_path) as fp:  # JSON file fits in memory
        payload = json.load(fp)
    rows = []
    for qid, q_payload in payload.items():
        question = (q_payload.get("question") or "").strip()
        answers = q_payload.get("answers", {})
        for aid, ans_payload in answers.items():
            article = (ans_payload.get("article") or "").strip()
            target = (ans_payload.get("answer_abs_summ") or "").strip()
            if not article or not target:
                continue  # skip incomplete rows
            prompt = (
                "Summarize the following clinical article to answer the patient question.\n"
                f"Question: {question}\n\n"
                f"Article:\n{article}"
            ).strip()
            rows.append(
                {
                    "uid": f"{qid}_{aid}",
                    "question": question,
                    "article": article,
                    "input_text": prompt,
                    "target_text": target,
                }
            )
    return rows

splits = {}
for name in ["train", "validation", "test"]:
    split_path = DATA_DIR / f"{name}.json"
    split_rows = load_split(split_path)
    splits[name] = split_rows
    print(f"{name.title()} examples: {len(split_rows)}")

dataset_dict = DatasetDict({k: Dataset.from_list(v) for k, v in splits.items()})
dataset_dict

In [ ]:
sample_row = dataset_dict["train"][0]
for key in ["uid", "question", "input_text", "target_text"]:
    preview = sample_row[key][:500]
    suffix = "..." if len(sample_row[key]) > len(preview) else ""
    print(f"\n--- {key} ---")
    print(preview + suffix)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Tokenizer max length: {tokenizer.model_max_length}")


def preprocess_function(batch):
    model_inputs = tokenizer(
        batch["input_text"],
        max_length=MAX_INPUT_TOKENS,
        truncation=True,
        padding="max_length",
    )
    labels = tokenizer(
        batch["target_text"],
        max_length=MAX_TARGET_TOKENS,
        truncation=True,
        padding="max_length",
    )

    label_ids = []
    for input_ids, attn_mask in zip(labels["input_ids"], labels["attention_mask"]):
        clean_ids = [token if mask == 1 else -100 for token, mask in zip(input_ids, attn_mask)]
        label_ids.append(clean_ids)

    model_inputs["labels"] = label_ids
    return model_inputs


remove_cols = dataset_dict["train"].column_names
tokenized_datasets = dataset_dict.map(
    preprocess_function,
    batched=True,
    remove_columns=remove_cols,
    desc="Tokenizing dataset",
)

tokenized_datasets


In [ ]:
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]
    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]
    preds = np.asarray(preds)
    if preds.ndim == 3:  # logits → token ids
        preds = np.argmax(preds, axis=-1)
    preds = preds.astype(np.int64, copy=False)
    preds = np.where(preds < 0, tokenizer.pad_token_id, preds)
    decoded_preds = tokenizer.batch_decode(preds.tolist(), skip_special_tokens=True)

    if isinstance(labels, tuple):
        labels = labels[0]
    labels = np.asarray(labels, dtype=np.int64)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels.tolist(), skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    rouge_result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
    )
    result = {k: round(v, 4) for k, v in rouge_result.items()}
    result["prediction_len"] = np.mean(
        [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    )
    return result


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
model.resize_token_embeddings(len(tokenizer))

label_smoothing = 0.1

import inspect

training_kwargs = dict(
    output_dir=str(OUTPUT_DIR / "flan_t5_finetuned"),
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_strategy="steps",
    logging_steps=25,
    save_strategy="epoch",
    save_total_limit=2,
    predict_with_generate=True,
    generation_max_length=MAX_TARGET_TOKENS,
    report_to=["none"],
    seed=SEED,
    label_smoothing_factor=label_smoothing,
    load_best_model_at_end=True,
    metric_for_best_model="rougeLsum",
    greater_is_better=True,
)

training_signature = inspect.signature(Seq2SeqTrainingArguments.__init__)
if "evaluation_strategy" in training_signature.parameters:
    training_kwargs["evaluation_strategy"] = "epoch"
elif "eval_strategy" in training_signature.parameters:
    training_kwargs["eval_strategy"] = "epoch"
else:
    print("⚠️ evaluation_strategy not supported in this transformers version; defaulting to no periodic eval.")

if "fp16" in training_signature.parameters:
    training_kwargs["fp16"] = torch.cuda.is_available()

training_args = Seq2SeqTrainingArguments(**training_kwargs)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("Trainer ready ✅")


In [ ]:
train_result = trainer.train()
trainer.save_model()
trainer.save_metrics("train", train_result.metrics)
trainer.save_state()
print("Training complete ✅")


In [ ]:
validation_metrics = trainer.evaluate(eval_dataset=tokenized_datasets["validation"], metric_key_prefix="eval")
trainer.save_metrics("eval", validation_metrics)
print("Validation metrics:", validation_metrics)

if "test" in tokenized_datasets:
    test_metrics = trainer.evaluate(eval_dataset=tokenized_datasets["test"], metric_key_prefix="test")
    trainer.save_metrics("test", test_metrics)
    print("Test metrics:", test_metrics)


In [ ]:
PREDICTION_DIR = OUTPUT_DIR / "flan_t5_predictions"
PREDICTION_DIR.mkdir(parents=True, exist_ok=True)


def chunk_list(items, size):
    for idx in range(0, len(items), size):
        yield items[idx : idx + size]


def generate_and_save_predictions(model, tokenizer, records, split_name, output_path, batch_size=4, max_new_tokens=MAX_TARGET_TOKENS):
    model.to(device)
    model.eval()
    rows = []
    for batch in chunk_list(records, batch_size):
        inputs = tokenizer(
            [row["input_text"] for row in batch],
            max_length=MAX_INPUT_TOKENS,
            truncation=True,
            padding=True,
            return_tensors="pt",
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            generated = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                num_beams=4,
                length_penalty=1.0,
            )
        decoded = tokenizer.batch_decode(generated, skip_special_tokens=True)
        for row, pred in zip(batch, decoded):
            rows.append(
                {
                    "uid": row["uid"],
                    "question": row["question"],
                    "article": row["article"],
                    "reference": row["target_text"],
                    "prediction": pred.strip(),
                }
            )
    with open(output_path, "w") as fp:
        json.dump(rows, fp, indent=2)
    print(f"Saved {len(rows)} predictions → {output_path}")


FINETUNED_VAL_PATH = PREDICTION_DIR / "finetuned_validation_predictions.json"
FINETUNED_TEST_PATH = PREDICTION_DIR / "finetuned_test_predictions.json"

generate_and_save_predictions(trainer.model, tokenizer, splits["validation"], "validation", FINETUNED_VAL_PATH)
generate_and_save_predictions(trainer.model, tokenizer, splits["test"], "test", FINETUNED_TEST_PATH)


In [ ]:
def compute_text_metrics(pred_path, prefix):
    with open(pred_path) as fp:
        payload = json.load(fp)
    predictions = [row["prediction"] for row in payload]
    references = [row["reference"] for row in payload]
    rouge_scores = rouge.compute(predictions=predictions, references=references, use_stemmer=True)
    bert_scores = bertscore.compute(predictions=predictions, references=references, lang="en")
    metrics = {f"{prefix}_" + k: round(v, 4) for k, v in rouge_scores.items()}
    metrics.update(
        {
            f"{prefix}_bertscore_precision": float(np.mean(bert_scores["precision"])),
            f"{prefix}_bertscore_recall": float(np.mean(bert_scores["recall"])),
            f"{prefix}_bertscore_f1": float(np.mean(bert_scores["f1"])),
        }
    )
    return metrics

finetuned_val_metrics = compute_text_metrics(FINETUNED_VAL_PATH, "finetuned_val")
finetuned_test_metrics = compute_text_metrics(FINETUNED_TEST_PATH, "finetuned_test")

print("Finetuned validation metrics:", json.dumps(finetuned_val_metrics, indent=2))
print("Finetuned test metrics:", json.dumps(finetuned_test_metrics, indent=2))


In [ ]:
BASELINE_DIR = OUTPUT_DIR / "flan_t5_baseline"
BASELINE_DIR.mkdir(parents=True, exist_ok=True)

baseline_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
baseline_model.to(device)
baseline_model.eval()

BASELINE_VAL_PATH = BASELINE_DIR / "baseline_validation_predictions.json"
BASELINE_TEST_PATH = BASELINE_DIR / "baseline_test_predictions.json"

generate_and_save_predictions(baseline_model, tokenizer, splits["validation"], "validation", BASELINE_VAL_PATH)
generate_and_save_predictions(baseline_model, tokenizer, splits["test"], "test", BASELINE_TEST_PATH)

baseline_val_metrics = compute_text_metrics(BASELINE_VAL_PATH, "baseline_val")
baseline_test_metrics = compute_text_metrics(BASELINE_TEST_PATH, "baseline_test")

print("Baseline validation metrics:", json.dumps(baseline_val_metrics, indent=2))
print("Baseline test metrics:", json.dumps(baseline_test_metrics, indent=2))


## Next steps
- Inspect the JSON prediction files saved under `outputs/` for downstream analysis or comparison with Gemma-2 results.
- Swap out hyperparameters (epochs, LR, max tokens) and re-run the fine-tuning cell to explore trade-offs.
- If you only need inference, skip the training cell and run the baseline generation cell directly.
